# Hyperparameter tuning by randomized-search

In the previous notebook, we showed how to use a grid-search approach to
search for the best hyperparameters maximizing the generalization performance
of a predictive model.

However, a grid-search approach has limitations. It does not scale well when
the number of parameters to tune increases. Also, the grid imposes a
regularity during the search which might miss better parameter
values between two consecutive values on the grid.

In this notebook, we present a different method to tune hyperparameters called
randomized search.

## Our predictive model

Let us reload the dataset as we did previously:

In [1]:
import pandas as pd

adult_census = pd.read_csv("../datasets/adult-census.csv")

We extract the column containing the target.

In [2]:
target_name = "class"
target = adult_census[target_name]
target

0         <=50K
1         <=50K
2          >50K
3          >50K
4         <=50K
          ...  
48837     <=50K
48838      >50K
48839     <=50K
48840     <=50K
48841      >50K
Name: class, Length: 48842, dtype: object

We drop from our data the target and the `"education-num"` column which
duplicates the information with `"education"` columns.

In [3]:
data = adult_census.drop(columns=[target_name, "education-num"])
data

,age,workclass,education,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,25,Private,11th,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States
1,38,Private,HS-grad,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States
2,28,Local-gov,Assoc-acdm,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States
3,44,Private,Some-college,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States
4,18,?,Some-college,Never-married,?,Own-child,White,Female,0,0,30,United-States
...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27,Private,Assoc-acdm,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States
48838,40,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States
48839,58,Private,HS-grad,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States
48840,22,Private,HS-grad,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States


Once the dataset is loaded, we split it into a training and testing sets.

In [4]:
from sklearn.model_selection import train_test_split

data_train, data_test, target_train, target_test = train_test_split(
    data, target, random_state=42
)

We create the same predictive pipeline as done for the grid-search section.

In [5]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import make_column_selector as selector

categorical_columns_selector = selector(dtype_include=object)
categorical_columns = categorical_columns_selector(data)

categorical_preprocessor = OrdinalEncoder(
    handle_unknown="use_encoded_value", unknown_value=-1
)
preprocessor = make_column_transformer(
    (categorical_preprocessor, categorical_columns),
    remainder="passthrough",
    force_int_remainder_cols=False,  # Silence a warning in scikit-learn v1.6.
)

In [6]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import Pipeline

model = Pipeline(
    [
        ("preprocessor", preprocessor),
        (
            "classifier",
            HistGradientBoostingClassifier(random_state=42, max_leaf_nodes=4),
        ),
    ]
)

model

,steps,"[('preprocessor', ...), ('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('ordinalencoder', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


## Tuning using a randomized-search

With the `GridSearchCV` estimator, the parameters need to be specified
explicitly. We already mentioned that exploring a large number of values for
different parameters quickly becomes untractable.

Instead, we can randomly generate the parameter candidates. Indeed, such
approach avoids the regularity of the grid. Hence, adding more evaluations can
increase the resolution in each direction. This is the case in the frequent
situation where the choice of some hyperparameters is not very important, as
for the hyperparameter 2 in the figure below.

![Randomized vs grid search](../figures/grid_vs_random_search.svg)

Indeed, the number of evaluation points needs to be divided across the two
different hyperparameters. With a grid, the danger is that the region of good
hyperparameters may fall between lines of the grid. In the figure such region
is aligned with the grid given that hyperparameter 2 has a weak influence.
Rather, stochastic search samples the hyperparameter 1 independently from the
hyperparameter 2 and find the optimal region.

The `RandomizedSearchCV` class allows for such stochastic search. It is used
similarly to the `GridSearchCV` but the sampling distributions need to be
specified instead of the parameter values. For instance, we can draw
candidates using a log-uniform distribution because the parameters we are
interested in take positive values with a natural log scaling (.1 is as close
to 1 as 10 is).

<div class="admonition note alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">Note</p>
<p class="last">Random search (with <tt class="docutils literal">RandomizedSearchCV</tt>) is typically beneficial compared to
grid search (with <tt class="docutils literal">GridSearchCV</tt>) to optimize 3 or more hyperparameters.</p>
</div>

We now optimize 3 other parameters in addition to the ones we optimized in
the notebook presenting the `GridSearchCV`:

* `l2_regularization`: it corresponds to the strength of the regularization;
* `min_samples_leaf`: it corresponds to the minimum number of samples required
  in a leaf;
* `max_bins`: it corresponds to the maximum number of bins to construct the
  histograms.

We recall the meaning of the 2 remaining parameters:

* `learning_rate`: it corresponds to the speed at which the gradient-boosting
  corrects the residuals at each boosting iteration;
* `max_leaf_nodes`: it corresponds to the maximum number of leaves for each
  tree in the ensemble.

<div class="admonition note alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">Note</p>
<p class="last"><tt class="docutils literal">scipy.stats.loguniform</tt> can be used to generate floating numbers. To generate
random values for integer-valued parameters (e.g. <tt class="docutils literal">min_samples_leaf</tt>) we can
adapt it as follows:</p>
</div>

In [7]:
from scipy.stats import loguniform


class loguniform_int:
    """Integer valued version of the log-uniform distribution"""

    def __init__(self, a, b):
        self._distribution = loguniform(a, b)

    def rvs(self, *args, **kwargs):
        """Random variable sample"""
        return self._distribution.rvs(*args, **kwargs).astype(int)

Now, we can define the randomized search using the different distributions.
Executing 10 iterations of 5-fold cross-validation for random parametrizations
of this model on this dataset can take from 10 seconds to several minutes,
depending on the speed of the host computer and the number of available
processors.

In [8]:
%%time
from sklearn.model_selection import RandomizedSearchCV

param_distributions = {
    "classifier__l2_regularization": loguniform(1e-6, 1e3),
    "classifier__learning_rate": loguniform(0.001, 10),
    "classifier__max_leaf_nodes": loguniform_int(2, 256),
    "classifier__min_samples_leaf": loguniform_int(1, 100),
    "classifier__max_bins": loguniform_int(2, 255),
}

model_random_search = RandomizedSearchCV(
    model,
    param_distributions=param_distributions,
    n_iter=10,
    cv=5,
    verbose=1,
)
model_random_search.fit(data_train, target_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


C:\Users\Aymen\miniconda3\envs\scikit-learn-course\Lib\site-packages\sklearn\compose\_column_transformer.py:975: FutureWarning: The parameter `force_int_remainder_cols` is deprecated and will be removed in 1.9. It has no effect. Leave it to its default value to avoid this warning.
  warnings.warn(
C:\Users\Aymen\miniconda3\envs\scikit-learn-course\Lib\site-packages\sklearn\compose\_column_transformer.py:975: FutureWarning: The parameter `force_int_remainder_cols` is deprecated and will be removed in 1.9. It has no effect. Leave it to its default value to avoid this warning.
  warnings.warn(
C:\Users\Aymen\miniconda3\envs\scikit-learn-course\Lib\site-packages\sklearn\compose\_column_transformer.py:975: FutureWarning: The parameter `force_int_remainder_cols` is deprecated and will be removed in 1.9. It has no effect. Leave it to its default value to avoid this warning.
  warnings.warn(
C:\Users\Aymen\miniconda3\envs\scikit-learn-course\Lib\site-packages\sklearn\compose\_column_transforme

CPU times: total: 3min 1s
Wall time: 25.7 s


,estimator,Pipeline(step...m_state=42))])
,param_distributions,"{'classifier__l2_regularization': <scipy.stats....0020685291400>, 'classifier__learning_rate': <scipy.stats....00206851D7390>, 'classifier__max_bins': <__main__.log...00206851D7750>, 'classifier__max_leaf_nodes': <__main__.log...0020685291550>, ...}"
,n_iter,10
,scoring,None
,n_jobs,None
,refit,True
,cv,5
,verbose,1
,pre_dispatch,'2*n_jobs'
,random_state,None
,error_score,nan


Then, we can compute the accuracy score on the test set.

In [9]:
accuracy = model_random_search.score(data_test, target_test)

print(f"The test accuracy score of the best model is {accuracy:.2f}")

The test accuracy score of the best model is 0.87


In [10]:
from pprint import pprint

print("The best parameters are:")
pprint(model_random_search.best_params_)

The best parameters are:
{'classifier__l2_regularization': np.float64(0.00018556942610406087),
 'classifier__learning_rate': np.float64(0.0837355107000601),
 'classifier__max_bins': np.int64(89),
 'classifier__max_leaf_nodes': np.int64(34),
 'classifier__min_samples_leaf': np.int64(1)}


We can inspect the results using the attributes `cv_results` as we did
previously.

In [11]:
# get the parameter names
column_results = [f"param_{name}" for name in param_distributions.keys()]
column_results += ["mean_test_score", "std_test_score", "rank_test_score"]

cv_results = pd.DataFrame(model_random_search.cv_results_)
cv_results = cv_results[column_results].sort_values(
    "mean_test_score", ascending=False
)


def shorten_param(param_name):
    if "__" in param_name:
        return param_name.rsplit("__", 1)[1]
    return param_name


cv_results = cv_results.rename(shorten_param, axis=1)
cv_results

,l2_regularization,learning_rate,max_leaf_nodes,min_samples_leaf,max_bins,mean_test_score,std_test_score,rank_test_score
8,0.000186,0.083736,34,1,89,0.861948,0.002612,1
1,472.280027,0.686319,23,6,99,0.861729,0.002938,2
7,5.464883,0.371876,138,64,18,0.851492,0.002081,3
9,0.000090,0.020944,11,5,19,0.846633,0.002944,4
3,0.001224,0.029721,134,47,6,0.831181,0.001961,5
2,0.000210,0.011287,7,27,131,0.817996,0.002312,6
5,0.000002,2.134525,142,3,104,0.776856,0.016953,7
0,121.549056,0.001238,37,1,2,0.758947,0.000013,8
6,0.000111,0.001286,64,2,13,0.758947,0.000013,8
4,0.000540,7.230426,75,25,252,0.671348,0.076454,10


Keep in mind that tuning is limited by the number of different combinations of
parameters that are scored by the randomized search. In fact, there might be
other sets of parameters leading to similar or better generalization
performances but that were not tested in the search. In practice, a randomized
hyperparameter search is usually run with a large number of iterations. In
order to avoid the computation cost and still make a decent analysis, we load
the results obtained from a similar search with 500 iterations.

In [12]:
model_random_search = RandomizedSearchCV(
    model, param_distributions=param_distributions, n_iter=500,
    n_jobs=2, cv=5)
model_random_search.fit(data_train, target_train)
cv_results =  pd.DataFrame(model_random_search.cv_results_)
cv_results.to_csv("../figures/randomized_search_results.csv")

C:\Users\Aymen\miniconda3\envs\scikit-learn-course\Lib\site-packages\sklearn\compose\_column_transformer.py:975: FutureWarning: The parameter `force_int_remainder_cols` is deprecated and will be removed in 1.9. It has no effect. Leave it to its default value to avoid this warning.
  warnings.warn(


In [13]:
cv_results = pd.read_csv(
    "../figures/randomized_search_results.csv", index_col=0
)

(
    cv_results[column_results]
    .rename(shorten_param, axis=1)
    .sort_values("mean_test_score", ascending=False)
)

,l2_regularization,learning_rate,max_leaf_nodes,min_samples_leaf,max_bins,mean_test_score,std_test_score,rank_test_score
123,0.000040,0.187247,9,26,129,0.870465,0.001437,1
191,0.032587,0.143899,13,4,218,0.870383,0.001925,2
455,0.549773,0.175749,9,2,193,0.870137,0.001713,3
63,0.000006,0.112519,17,15,175,0.870028,0.001755,4
417,2.647649,0.089187,20,4,188,0.869974,0.002413,5
...,...,...,...,...,...,...,...,...
282,0.000065,8.890461,2,5,7,0.283476,0.005123,493
421,0.000006,5.863487,3,53,114,0.283476,0.005123,493
177,0.000115,8.161579,7,6,191,0.281702,0.016948,498
141,0.011446,2.232862,2,1,6,0.271818,0.025208,499


In this case the top performing models have test scores with a high overlap
between each other, meaning that indeed, the set of parameters leading to the
best generalization performance is not unique.


In this notebook, we saw how a randomized search offers a valuable alternative
to grid-search when the number of hyperparameters to tune is more than two. It
also alleviates the regularity imposed by the grid that might be problematic
sometimes.

In the following, we will see how to use interactive plotting tools to explore
the results of large hyperparameter search sessions and gain some insights on
range of parameter values that lead to the highest performing models and how
different hyperparameter are coupled or not.